In [5]:
import pandas as pd
import pm4py
from tqdm import tqdm
import os
import numpy as np

In [32]:
#  ALREADY RAN
for file in os.listdir('../data/train_test_data/'):
    in_path = '../data/train_test_data/' + file
    out_path = '../data/' + file.replace('.xes', '.csv')
    log = pm4py.read_xes(in_path)
    df = pm4py.convert_to_dataframe(log)
    df.to_csv(out_path, index=False)

parsing log, completed traces ::   0%|          | 0/8344 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/3001 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/751 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/13782 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/2087 [00:00<?, ?it/s]

parsing log, completed traces ::   0%|          | 0/3446 [00:00<?, ?it/s]

In [6]:
# declined_train = pd.read_csv('../data/declined_train.csv')
declined_test = pd.read_csv('../data/declined_test.csv', index_col=0).reset_index(drop=True)
print(len(declined_test['case:concept:name'].unique()))

declined_train = pd.read_csv('../data/declined_train.csv', index_col=0).reset_index(drop=True)
print(len(declined_train['case:concept:name'].unique()))

751
3001


In [7]:
approved_train = pd.read_csv('../data/approved_train.csv', index_col=0).reset_index(drop=True)
approved_test = pd.read_csv('../data/approved_test.csv', index_col=0).reset_index(drop=True)



cancelled_train = pd.read_csv('../data/cancelled_train.csv', index_col=0).reset_index(drop=True)
cancelled_test = pd.read_csv('../data/cancelled_test.csv', index_col=0).reset_index(drop=True)

In [8]:
from random import sample
train = pd.concat([declined_train, approved_train[approved_train['case:concept:name'].isin(sample(list(approved_train['case:concept:name'].unique()), 3000))], cancelled_train[cancelled_train['case:concept:name'].isin(sample(list(cancelled_train['case:concept:name'].unique()), 3000))]])
train['event_w_lifecycle'] = train['concept:name'] + ' ' + train['lifecycle:transition']
train 

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,event_w_lifecycle
0,Created,User_1,A_Create Application,Application,Application_1691306052,complete,2016-01-01 10:16:11.500000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Create Application complete
1,statechange,User_1,A_Submitted,Application,ApplState_284636842,complete,2016-01-01 10:16:11.549000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,A_Submitted complete
2,Created,User_1,W_Handle leads,Workflow,Workitem_831373279,schedule,2016-01-01 10:16:11.740000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Handle leads schedule
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1299098074,withdraw,2016-01-01 10:17:31.573000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Handle leads withdraw
4,Created,User_1,W_Complete application,Workflow,Workitem_1703931302,schedule,2016-01-01 10:17:31.584000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Home improvement,New credit,Application_1691306052,10000.0,W_Complete application schedule
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219131,Obtained,User_43,W_Call after offers,Workflow,Workitem_162257588,start,2016-11-12 09:31:06.232000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,W_Call after offers start
219132,statechange,User_43,A_Complete,Application,ApplState_1435859997,complete,2016-11-12 09:31:06.234000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,A_Complete complete
219133,statechange,User_43,A_Cancelled,Application,ApplState_860885554,complete,2016-11-12 09:35:55.142000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,A_Cancelled complete
219134,statechange,User_43,O_Cancelled,Offer,OfferState_2011904028,complete,2016-11-12 09:35:55.161000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119,Not speficied,New credit,Application_1583440896,50000.0,O_Cancelled complete


In [9]:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#reshaping-dummies to create the the categorical encoding
# https://stackoverflow.com/questions/49161120/pandas-python-set-value-of-one-column-based-on-value-in-another-column to assign values of the result DF
# https://stackoverflow.com/questions/71426679/cumulative-sum-of-time-from-timestamps-in-pandas for cumulative time for boris

In [10]:
org_resource_cols = list(train['org:resource'].unique())
event_cols = list(train['event_w_lifecycle'].unique())
rest_cols = ['FirstWithdrawalAmount',
       'NumberOfTerms', 'Accepted', 'MonthlyCost', 'Selected', 'CreditScore',
       'OfferedAmount', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount']
target_cols = ['first_timestamp', 'trace_duration', 'case_outcome']

res_cols = org_resource_cols + event_cols + rest_cols + target_cols


In [16]:
tester = train[train['case:concept:name'] == 'Application_1583440896']
tester

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,FirstWithdrawalAmount,NumberOfTerms,Accepted,MonthlyCost,Selected,CreditScore,OfferedAmount,OfferID,case:LoanGoal,case:ApplicationType,case:concept:name,case:RequestedAmount,event_w_lifecycle
219119,Created,User_1,A_Create Application,Application,Application_1583440896,complete,2016-11-12 09:23:42.583000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,A_Create Application complete
219120,statechange,User_1,A_Submitted,Application,ApplState_2137857486,complete,2016-11-12 09:23:43.703000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,A_Submitted complete
219121,Created,User_1,W_Handle leads,Workflow,Workitem_1291517221,schedule,2016-11-12 09:23:43.863000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,W_Handle leads schedule
219122,Deleted,User_1,W_Handle leads,Workflow,Workitem_2127860955,withdraw,2016-11-12 09:24:21.906000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,W_Handle leads withdraw
219123,Created,User_1,W_Complete application,Workflow,Workitem_956641277,schedule,2016-11-12 09:24:21.912000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,W_Complete application schedule
219124,statechange,User_1,A_Concept,Application,ApplState_1942002841,complete,2016-11-12 09:24:21.915000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,A_Concept complete
219125,statechange,User_43,A_Accepted,Application,ApplState_1708537144,complete,2016-11-12 09:29:28.397000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not speficied,New credit,Application_1583440896,50000.0,A_Accepted complete
219126,Created,User_43,O_Create Offer,Offer,Offer_1248413119,complete,2016-11-12 09:30:59.387000+00:00,40000.0,60.0,False,738.85,False,0.0,40000.0,NaN,Not speficied,New credit,Application_1583440896,50000.0,O_Create Offer complete
219127,statechange,User_43,O_Created,Offer,OfferState_1753374182,complete,2016-11-12 09:30:59.994000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119,Not speficied,New credit,Application_1583440896,50000.0,O_Created complete
219128,statechange,User_43,O_Sent (mail and online),Offer,OfferState_1580560695,complete,2016-11-12 09:31:06.212000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Offer_1248413119,Not speficied,New credit,Application_1583440896,50000.0,O_Sent (mail and online) complete


In [70]:
def aggregate_df(df, res_cols):
    """ 
    Aggregate the df of current events in the case

    Output: 
        result -> could be a pandas series
    """
    res_dict = dict.fromkeys(res_cols)
    
    for row in df.to_dict('records'):

        # user variable assign
        user = row['org:resource']
        # first mention, turn it numeric
        if not res_dict[user]: res_dict[user] = 1
        else: res_dict[user] += 1
    
        # event variable assign
        event = row['concept:name'] + ' ' + row['lifecycle:transition']
        if not res_dict[event]: res_dict[event] = 1
        else: res_dict[event] += 1

        # case level cols
        rest_cols = ['FirstWithdrawalAmount',
       'NumberOfTerms', 'Accepted', 'MonthlyCost', 'Selected', 'CreditScore',
       'OfferedAmount', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount'] 

        for col in rest_cols:
            if res_dict[col] is None:
                res_dict[col] = row[col]
            elif np.isnan(res_dict[col]):
                res_dict[col] = row[col]
        
        # time stuff
        #keeping the first timestamp of the case for calculation purposes
        if not res_dict['first_timestamp']: res_dict['first_timestamp'] = row['time:timestamp']

        # trace duration in seconds
        res_dict['trace_duration'] = (np.datetime64(row['time:timestamp']) - np.datetime64(res_dict['first_timestamp'])).item().total_seconds()

    return res_dict

pd.DataFrame.from_dict([aggregate_df(tester, res_cols)])["MonthlyCost"][0]

/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_2353/3590134581.py:39: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  res_dict['trace_duration'] = (np.datetime64(row['time:timestamp']) - np.datetime64(res_dict['first_timestamp'])).item().total_seconds()


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [71]:
def add_to_aggregate(result, df_row):
    """ 
    When a new event happens, add the event info to the current aggregated result.

    Input: 
         result: the current aggregated result
         df_row: pandas df row representing the new event
    Output:
        result: the new aggregated result
    """
    last_row = result[-1].copy()
    new_event = df_row.to_dict()
    new_row = last_row

    # user variable assign
    user = new_event['org:resource']
    # first mention, turn it numeric
    if not new_row[user]: new_row[user] = 1
    else: new_row[user] += 1

    # event variable assign
    event = new_event['concept:name'] + ' ' + new_event['lifecycle:transition']
    if not new_row[event]: new_row[event] = 1
    else: new_row[event] += 1

    # case level cols
    rest_cols = ['FirstWithdrawalAmount',
    'NumberOfTerms', 'Accepted', 'MonthlyCost', 'Selected', 'CreditScore',
    'OfferedAmount', 'case:LoanGoal', 'case:ApplicationType', 'case:RequestedAmount'] 

    for col in rest_cols:
        if np.isnan(new_row[col]): 
            new_row[col] = new_event[col]
        elif new_row[col] is None: 
            new_row[col] = new_event[col]
    
    # time stuff
    # trace duration in seconds
    new_row['trace_duration'] = (np.datetime64(new_event['time:timestamp']) - np.datetime64(new_row['first_timestamp'])).item().total_seconds()

    return new_row

In [72]:
def create_prefix_part2(cases_df, res_cols, end_event, start_event='A_Accepted'):
    
    app_id_list = list(cases_df['case:concept:name'].unique())

    # TODO:
    # create a return df
    return_list = []

    # extracting prefix for each application
    for app_id in app_id_list:
        
        events_app = cases_df.loc[cases_df['case:concept:name'] == app_id]
        events_app.reset_index(drop=True, inplace=True)

        cur_id = starting_row_id = events_app.loc[events_app['concept:name'] == 'A_Accepted'].index[0]
        pre_events = events_app.iloc[:starting_row_id]
        # TODO: 
        # aggregate events_app from row 0 to starting_row_id
        return_list.append(aggregate_df(pre_events, res_cols=res_cols))
        
        ending_row_id = events_app.loc[events_app['concept:name'] == end_event].index[0] -1
        cur_id += 1
        
        while cur_id < ending_row_id:
            new_row = events_app.iloc[cur_id]
            # TODO: 
            # add new event row info to the aggregated result
            return_list.append(add_to_aggregate(return_list, new_row))

            # Update the return_df -> add new row
            # target y: end_event

            cur_id += 1

    result = pd.DataFrame.from_dict(return_list)
    result['case_outcome'] = end_event
    return result

In [73]:
create_prefix_part2(tester, res_cols, 'A_Cancelled')

/var/folders/9l/kwd1671542z9tyvsgbp6mmtr0000gn/T/ipykernel_2353/3590134581.py:39: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  res_dict['trace_duration'] = (np.datetime64(row['time:timestamp']) - np.datetime64(res_dict['first_timestamp'])).item().total_seconds()


TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [74]:
type(tester['CreditScore'][219119])

numpy.float64